Theorems (or conjectures) for the theory of <a class="ProveItLink" href="theory.ipynb">proveit.physics.quantum.circuits</a>
========

In [ ]:
import proveit
# Prepare this notebook for defining the theorems of a theory:
%theorems_notebook # Keep this at the top following 'import proveit'.
from proveit import defaults
defaults.automation = True # Hack for ExprRange simplification to go through

from proveit import a, b, i, j, k, l, m, n, p, t, A, P, Q, U
from proveit import ExprRange, var_range, IndexedVar
from proveit.core_expr_types import (
    a_1_to_k, b_1_to_k, c_1_to_k, A_1_to_j, A_1_to_m, B_1_to_k, B_1_to_n, C_1_to_l, D_1_to_m, U_1_to_m)
from proveit.core_expr_types.expr_arrays import (
    A11_to_Akl, A11_to_Akm, B11_to_Bkm, B11_to_Bkn, C11_to_Clm, C11_to_Ckm, D11_to_Dkn,
    R11_to_Rkm, S11_to_Skm, S11_to_Skn, T11_to_Tlm, T11_to_Tkm, U11_to_Ukn, V11_to_Vkm)
from proveit.logic import And, Implies, Forall, InSet, Equals
from proveit.numbers import Natural, NaturalPos, Interval, Exp, one, two, exp2pi_i
from proveit.linear_algebra import SU, ScalarMult
from proveit.physics.quantum import SPACE, var_ket_psi, varphi, m_ket_domain, var_ket_u
from proveit.physics.quantum import Qmult
from proveit.physics.quantum.circuits import (
    circuit_Am, circuit_Bn, circuit_Bk, circuit_Dm,
    circuit_AjBkCl, circuit_AjDmCl,
    circuit_aUb, circuit_aU, circuit_b,
    circuit_Akm, circuit_Bkm, circuit_Bkn, circuit_Ckm, circuit_Dkn,
    circuit_permuted_Akm, circuit_permuted_Bkn,
    circuit_AkClm, circuit_BkClm, no_1tok_in_Ts)
    #circuit_permuted_Akm, circuit_permuted_Bkn)
from proveit.physics.quantum.circuits import (
    QcircuitEquiv, Gate, MultiQubitElem, Input, Output,
    phase_kickback_circuit, phase_kickback_on_register_circuit)
#from proveit.physics.quantum.circuit import CircuitEquiv
#from proveit.physics.quantum.circuits import (
#    circuit_aUVc, circuit_aUb, circuit_bVc, 
#    circuit_A_detailed, circuit_B_detailed, permuted_circuit_A, permuted_circuit_B,
#    circuit_B, circuit_D, circuit_ABCvert, circuit_ADCvert
#)

In [ ]:
defaults.automation = False # Hack for ExprRange simplification to go through
%begin theorems
defaults.automation = True # Hack for ExprRange simplification to go through
print("We need automation to simplify ExprRanges in Qcircuits")

### MultiQubitElem reductions

In [ ]:
# for use in reducing a MultiQubitElem to an empty space within a Circuit.
empty_multi_qubit_gate_reduction = Equals(MultiQubitElem(SPACE, []), SPACE)

In [ ]:
# for use in reducing a MultiQubitElem to a gate within a Circuit.
unary_multi_qubit_gate_reduction = Forall(
    U, Forall(m, Equals(MultiQubitElem(Gate(U), [m]), 
                        Gate(U)), 
              domain=NaturalPos))

In [ ]:
# for use in reducing a MultiQubitElem to a Input within a Circuit.
unary_multi_qubit_input_reduction = Forall(
    var_ket_psi, Forall(m, Equals(MultiQubitElem(Input(var_ket_psi), [m]), 
                                  Input(var_ket_psi)), 
                        domain=NaturalPos))

In [ ]:
# for use in reducing a MultiQubitElem to a Input within a Circuit.
unary_multi_qubit_output_reduction = Forall(
    var_ket_psi, Forall(m, Equals(MultiQubitElem(Output(var_ket_psi), [m]), 
                                  Output(var_ket_psi)), 
                        domain=NaturalPos))

### Circuit equivalences and uses

Circuits are equivalent when the function in the same way -- they have the same output for any possible input.

Circuits are equivalent when they are the same except for a temporal section which is equivalent.  The up/down arrows denote entire columns which may be instantiated with `ExprTuple`s.

In [ ]:
circuit_equiv_temporal_sub = Forall(
    (j, k, l, m), Forall(
        (A_1_to_j, B_1_to_k, C_1_to_l, D_1_to_m),
        Implies(QcircuitEquiv(circuit_Bk, circuit_Dm),
                QcircuitEquiv(circuit_AjBkCl, circuit_AjDmCl).with_wrap_before_operator())
        .with_wrap_after_operator()),
    domain=Natural)

If a quantum circuit is "true", meaning that its inputs and outputs have all been specified and are consistent, then an equivalent quantum circuit is also true.  Note that a circuit cannot be provably "true" if it has any non-specified inputs or outputs.

In [ ]:
rhs_via_equiv = Forall(
    (k, m, n), Forall(
        (A_1_to_m, B_1_to_n),
        Implies(And(circuit_Am, QcircuitEquiv(circuit_Am, circuit_Bn)),
                circuit_Bn).with_wrap_before_operator()))

In [ ]:
lhs_via_equiv = Forall(
    (k, m, n), Forall(
        (A_1_to_m, B_1_to_n),
        Implies(And(circuit_Bn, QcircuitEquiv(circuit_Am, circuit_Bn)),
                circuit_Am).with_wrap_before_operator()))

A quantum circuit applied to some input is equivalent to its output in the form of a circuit input.  That is, if a circuit produces some output, it is interchangable with that output when fed as an input in a broader circuit.

In [ ]:
circuit_output_equiv = Forall(
    (k, m), Forall(
        (a, b, U_1_to_m),
        Implies(circuit_aUb, 
                QcircuitEquiv(circuit_aU, circuit_b))),
    domain=NaturalPos)

Circuit equivalence is preserved under the permutation of qubit row indices.

In [ ]:
# This version allows you to prove any QcircuitEquiv from any other.
# This is wrong, of course, but a useful placeholder while we work on the proper
# version which may require some core changes w.r.t. instantiations parameter ranges.
circuit_equiv_qubit_permutation_cheater = Forall(
    (k, m, n),
    Forall((A11_to_Akm, B11_to_Bkn, C11_to_Ckm, D11_to_Dkn),
           Forall((R11_to_Rkm, S11_to_Skn, T11_to_Tkm, U11_to_Ukn),
                  Equals(QcircuitEquiv(circuit_Akm, circuit_Bkn),
                         QcircuitEquiv(circuit_Ckm, circuit_Dkn))
                  .with_wrap_after_operator()).with_wrapping())
    .with_wrapping(),
    domain=NaturalPos)

In [ ]:
from proveit import Literal, Function
circuit_equiv_qubit_permutation = Forall(
    (k, m, n),
    Forall(p,
           Forall((A11_to_Akm, R11_to_Rkm, B11_to_Bkn, S11_to_Skn),
                  Equals(QcircuitEquiv(circuit_Akm, circuit_Bkn),
                         QcircuitEquiv(circuit_permuted_Akm, circuit_permuted_Bkn))
                  .with_wrap_after_operator()).with_wrapping(),
           domain=Function(Literal('Perm', latex_format=r'\\textrm{Perm}'),
                           Interval(one, k))), # TODO, ADD Perm\n",
    domain=NaturalPos)

In [ ]:
%%latex
$(x_1, ..., x_n) \\
p^{\leftarrow}((x_1, ..., x_n)) = (p^{-1}(x_1), ..., p^{-1}(x_n)) \\
p^{\leftarrow}(S) = \{y~|~p(y) \in S\}
$
U on (3, 1, 2)

Circuits are equivalent when they are the same except for a top section which is equivalent as long as there are no multi-gate that cross the top and bottom sections and there is no control or swap across the sections.  There could be a control in the top section with a target in the bottom section as long as circuit equivalence accounts for control on external targets as part of the output that must all be consistent for each possible input.

In [ ]:
circuit_equiv_top_sub= Forall(
    (k, l, m),
    Forall((A11_to_Akm, B11_to_Bkm, C11_to_Clm),
           Forall((R11_to_Rkm, S11_to_Skm, T11_to_Tlm),
                  Implies(QcircuitEquiv(circuit_Akm, circuit_Bkm),
                          QcircuitEquiv(circuit_AkClm, circuit_BkClm))
                  .with_wrap_after_operator(),
                  conditions=no_1tok_in_Ts)
           .with_wrap_before_condition().with_wrapping())
    .with_wrapping(),
    domain=NaturalPos)

### Useful circuit truths

In [ ]:
defaults.assumptions = [InSet(m, NaturalPos)]
phase_kickback = Forall(
    m, Forall(
        U, Forall(
            var_ket_u, Forall(
                varphi, phase_kickback_circuit,
                condition=Equals(Qmult(U, var_ket_u,),
                                 ScalarMult(exp2pi_i(varphi), var_ket_u))),
            domain=m_ket_domain),
        domain=SU(Exp(two, m))),
    domain=NaturalPos)

In [ ]:
from proveit.numbers import greater, subtract, zero
defaults.assumptions = [InSet(t, NaturalPos), greater(t, two), greater(subtract(t, one), zero)]
phase_kickbacks_on_register = Forall(
    (m, t), Forall(
        var_range(U, one, t), Forall(
            var_ket_u, Forall(
                var_range(varphi, one, t), phase_kickback_on_register_circuit,
                conditions=ExprRange(i, Equals(Qmult(IndexedVar(U, i), var_ket_u),
                                               ScalarMult(exp2pi_i(IndexedVar(varphi, i)), 
                                                          var_ket_u)),
                                     one, t)).with_wrapping(),
            domain=m_ket_domain),
        domain=SU(Exp(two, m))),
    domain=NaturalPos)

In [ ]:
%end theorems

In [ ]:
# sub_circuit_inputs = Forall(
#     (k, m, n),
#     Forall(
#         (a_1_to_k, b_1_to_k, c_1_to_k, U11_to_Ukm, V11_to_Vkm, R11_to_Rkm, S11_to_Skm),
#         Implies(And(circuit_aUVc, circuit_aUb),
#                circuit_bVc).with_wrap_after_operator()),
#     domain=NaturalPos)

In [ ]:
# sub_circuit_inputs.instance_expr.instance_expr

In [ ]:
# qubit_permutation = Forall(
#     (P, Q), Forall(
#         (k, m, n),
#         Forall(
#             (A11_to_Akm, R11_to_Rkm, B11_to_Bkn, S11_to_Skn),
#             Implies(CircuitEquiv(circuit_A_detailed, circuit_B_detailed),
#                     CircuitEquiv(permuted_circuit_A, permuted_circuit_B)).with_wrap_after_operator()),
#         domain=NaturalPos))

In [ ]:
# qubit_permutation.instance_expr.instance_expr.instance_expr

In [ ]:
# qubit_range_circuit_substitution = Forall(
#     (k, l, m, n), Forall(
#         (A11_to_Akl, B11_to_Bkm, C11_to_Ckn, D11_to_Dkm),
#         Implies(CircuitEquiv(circuit_B, circuit_D),
#                 CircuitEquiv(circuit_ABCvert, circuit_ADCvert)).with_wrap_after_operator()),
#     domain=NaturalPos)

In [ ]:
# qubit_range_circuit_substitution.instance_expr.instance_expr

In [ ]:
# %end theorems